In [ ]:
# STL
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
import logging
# 3rd Party
import torch
from transformers import AutoTokenizer
# Local
from gatbert.data import parse_graph_tsv, make_file_parser, make_collate_fn
from gatbert.graph_sample import GraphSample, Edge
from gatbert.constants import Stance
from gatbert.utils import batched, change_log_level

In [2]:
pretrained_model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, use_fast=True, add_prefix_space=True)

In [3]:
parse_fn = make_file_parser('graph', tokenizer)
collate_fn = make_collate_fn('graph', tokenizer)

In [ ]:
sample_gen = parse_fn('scrap.tsv')
sample_gen = batched(sample_gen, 4)
sample_gen = map(collate_fn, sample_gen)
for batch in sample_gen:
    print(*[t.shape for t in batch.values()])


In [ ]:
graph_gen = parse_graph_tsv('scrap.tsv')
graph_gen = map(lambda sample: sample.encode(tokenizer), graph_gen)
# graph_gen = batched(graph_gen, 4)
# graph_gen = map(GraphSample.collate, graph_gen)
with change_log_level("transformers.tokenization_utils_base"):
    for batch in graph_gen: